#### Recall: Nesting Example from last time

```sql
SELECT  Actor
FROM    Movie
WHERE   title IN (
                    SELECT  title
                    FROM    Movie
                    WHERE   director = "Bertolucci"
                )
```

We can actually eliminate nesting:

```sql
SELECT  m1.actor
FROM    Movie m1, Movie m2
WHERE   m1.title = m2.title AND
        m2.director = "Bertolucci"
```

Piazza post noted that the nesting example was overcomplicated and ran through the table twice, was just for showcasing nested queries

Q: why do we need m1 and m2?
A: we need to diambiguate between tuples w/ similar titles (otherwise would refer to the same exact tuple)

Also it would suffice to just do

```sql
SELECT  DISTINCT actor
FROM    Movie
WHERE   director = "Bertolucci"
```

#### Question

Can we always eliminate nesting?

Queries involving nesting but **no negation** can always be unnested in contrast to queries with nesting and negation

Negation in the WHERE clause necessitates nesting

## Correlated Nested Queries

If condition in WHERE clause of *nestED query* references an attribute of a relation declared in the *outer query*, the two queries are said to be correlated

Result of a correlated nested query may be different for each tuple (or combinations of tuples) of the relation(s) the outer query

EX: Retrieve the name of each employee who has a dependent with the same first name as the employee

```sql
SELECT  E.FNAME, E.LNAME
FROM    EMPLOYEE E
WHERE   E.SSN IN (
    SELECT  ESSN
    FROM    DEPENDENT
    WHERE   ESSN=E.SSN
        AND E.FNAME=DEPENDENT_NAME
)
```

> NOTE: The nestd query gets recomputed each time due to correlation (it depends on `E.SSN`). As opposed to the movie by Bertolucci being the same each time and not needing to be recomputed

Can also be flattened by concatenating the FROM clauses

```sql
SELECT  E.FNAME, E.LNAME
FROM    EMPLOYEE E, DEPENDENT D
WHERE   E.SSN=D.ESSN AND 
        E.FNAME=D.DEPENDENT_NAME
```

membership test made expicit instead of implicit

> NOTE: nesting might not preserve multiplicity. 
 
The nested version of this goes through each Employee just once, even if multiple dependents from the nested query satisfy the condition and so the outer query, which only runs once, would only return the Employee once - even if multiple dependents shared the employee's first name.

The flattened one, on the other hand, iterates Employees alongside Dependents (think of |E| x |D| input domain as opposed to the nested queries |E| domain against the conditional |D| nested query). Thus it'll return the Employee for each dependent that shares their first name.

## Simple use of NOT IN

Ex: Find all movies in which Hitchcock **does not** act

```sql
SELECT  title FROM Movie
WHERE   title NOT IN
    (
        SELECT  title FROM Movie
        WHERE   actor = 'Hitchcock'
    )
```

Notice: the lack of correlation. So the nested query will compute the same way each time.

Outer loop iterates movies and selects those whose titles contained within the container of titles w/o Hitchcock as an actor.

#### Q: Why can't you flatten it like this?

Natural tendency to try to flatten it like so...

```sql
SELECT  m1.title
FROM    Movie m1, Movie m2
WHERE   NOT (m1.title = m2.title)       -- WHERE    m1.title <> m2.title
        AND m2.actor = 'Hitchcock'
```

Think of why this doesn't work? (It's more than just a multiplicity diff too!)

Flattened version will be LESS selective than the nested version, now think of why...

#### A: slightly counterintuitive way in which data is modeled in tables

Outer iteration of m1 through Movies
- Iterate m2 through Movies until 
  - a distinct title (m1.title <> m2.title) is found 
    - AND (m2.actor = 'Hitchcock')
      - This will JUMP to conclusions and break the nested loop early
        as soon as the first actor isn't Hitchcock, it'll return the title
        but Hitchcock could very well have been in another tuple

Tl;dr query on bottom will not wait, it jumps the gun due to interleaved loops. 